In [46]:
import sys
sys.path.append("/vtk/build/") #

In [47]:
import numpy as np

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot

# Define temporal parameters
t = 0 # Start time
T = 2.0 # Final time
num_steps = 61     
dt = T / num_steps # time step size

# Define mesh
nx, ny = 50, 50
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([-2, -2]), np.array([2, 2])], 
                               [nx, ny], mesh.CellType.triangle)
V = fem.FunctionSpace(domain, ("CG", 1))

In [48]:
# Create initial condition
alpha = 6          # parameter alpha
beta = 1.2         # parameter beta
def initial_condition(x, t=0):
    return 1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t
    # return np.exp(-a*(x[0]**2+x[1]**2))
u_n = fem.Function(V)
u_n.name = "u_n"
u_n.interpolate(initial_condition)

# Create boundary condition
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)

In [49]:
xdmf = io.XDMFFile(domain.comm, "heat_equation.xdmf", "w")
xdmf.write_mesh(domain)

# Define solution variable, and interpolate initial solution for visualization in Paraview
uh = fem.Function(V)
uh.name = "uh"
uh.interpolate(initial_condition)
xdmf.write_function(uh, t)

In [50]:
import ufl
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
# f = fem.Constant(domain, PETSc.ScalarType(0))
f = fem.Constant(domain,PETSc.ScalarType(beta - 2 - 2*alpha))
# a = u *v * ufl.dx + dt* ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx 

a = ufl.inner(u,v)*ufl.dx + ufl.inner(ufl.grad(u), ufl.grad(v))* dt * ufl.dx

L = ufl.inner((u_n + dt * f), v) * ufl.dx


In [51]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [52]:
A = fem.petsc.assemble_matrix(bilinear_form, bcs=[bc])
A.assemble()
b = fem.petsc.create_vector(linear_form)

In [53]:
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

In [54]:
import pyvista
pyvista.set_jupyter_backend("ipygany")

grid = pyvista.UnstructuredGrid(*plot.create_vtk_mesh(V))

def plot_function(t, uh):
    """
    Create a figure of the concentration uh warped visualized in 3D at timet step t.
    """
    p = pyvista.Plotter()
    # Update point values on pyvista grid
    grid.point_data[f"u({t})"] = uh.x.array.real
    # Warp mesh by point values
    warped = grid.warp_by_scalar(f"u({t})", factor=1.5)

    # Add mesh to plotter and visualize in notebook or save as figure
    actor = p.add_mesh(warped)
    if not pyvista.OFF_SCREEN:
       p.show()
    else:
        pyvista.start_xvfb()
        figure_as_array = p.screenshot(f"diffusion_{t:.2f}.png")
        # Clear plotter for next plot
        p.remove_actor(actor)
plot_function(0, uh)

AppLayout(children=(VBox(children=(HTML(value='<h3>u(0)</h3>'), Dropdown(description='Colormap:', options={'Br…

In [56]:
for i in range(num_steps):
    t += dt

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    fem.petsc.assemble_vector(b, linear_form)
    
    # Apply Dirichlet boundary condition to the vector
    fem.petsc.apply_lifting(b, [bilinear_form], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(b, [bc])

    # Solve linear problem
    solver.solve(b, uh.vector)
    uh.x.scatter_forward()

    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh.x.array

    # Write solution to file
    xdmf.write_function(uh, t)
    # Plot every 15th time step
    if i % 15 == 0:
        plot_function(t, uh)

xdmf.close()

AppLayout(children=(VBox(children=(HTML(value='<h3>u(2.032786885245903)</h3>'), Dropdown(description='Colormap…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(2.524590163934428)</h3>'), Dropdown(description='Colormap…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(3.0163934426229533)</h3>'), Dropdown(description='Colorma…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(3.5081967213114784)</h3>'), Dropdown(description='Colorma…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(4.0000000000000036)</h3>'), Dropdown(description='Colorma…